# 連合学習

## 必要なライブラリをインストール・インポート

In [ ]:
!pip install -q flwr[simulation]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

import flwr as fl
from flwr.common import Metrics

import multiprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import OrderedDict
from typing import List, Tuple, Dict, Optional
from __future__ import annotations

## CSVファイルの読み込み

In [ ]:
dataset_df = pd.read_csv("dataset.csv")
dataset_df.head()

,distance,width,speed,crowd_level,time
0,120.400000,1.0,1.4,1,86
1,103.549615,1.0,1.7,1,61
2,95.000000,1.0,1.0,1,95
3,85.000000,1.0,1.0,1,85
4,62.000000,1.0,1.0,1,62


## データセットの整形

In [ ]:
# "crowd_level"を文字列型に変換
dataset_df["crowd_level"] = dataset_df["crowd_level"].astype(str)
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   distance     10000 non-null  float64
 1   width        10000 non-null  float64
 2   speed        10000 non-null  float64
 3   crowd_level  10000 non-null  object 
 4   time         10000 non-null  int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 390.8+ KB


In [ ]:
# One-Hot Encoding
dataset_df = pd.get_dummies(dataset_df, columns=["crowd_level"], drop_first=True)
dataset_df

,distance,width,speed,time,crowd_level_2,crowd_level_3,crowd_level_4,crowd_level_5
0,120.400000,1.0,1.4,86,0,0,0,0
1,103.549615,1.0,1.7,61,0,0,0,0
2,95.000000,1.0,1.0,95,0,0,0,0
3,85.000000,1.0,1.0,85,0,0,0,0
4,62.000000,1.0,1.0,62,0,0,0,0
...,...,...,...,...,...,...,...,...
9995,26.324500,3.0,1.4,27,0,0,0,1
9996,26.324500,3.0,1.1,27,0,0,0,1
9997,26.000000,3.0,1.4,21,0,0,0,1
9998,25.856858,3.0,1.7,22,0,0,0,1


## 変数の準備

In [ ]:
NUM_CLIENTS = 100  # クライアント数
EPOCHS = int(3000 / NUM_CLIENTS)  # 各クライアントのエポック数
ROUNDS = 30        # ラウンド数
BATCH_SIZE = 1     # バッチサイズ
is_random = False  # データセットの分割（True:ランダム, False:歩行速度別）

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


## データセットの準備

In [ ]:
# ランダムにデータセットを分割
def load_datasets_random():
    """
    データセットをロード
    """
    # 説明変数と目的変数に分割
    X = dataset_df.drop(["distance", "time"], axis=1).values
    y = ((dataset_df["time"] / dataset_df["distance"]) - (1 / dataset_df["speed"])).values

    # NumPy配列をPyTorchテンソルに変換
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    # サイズを(N, 1)に変更
    y = y.view(-1, 1)

    # 訓練データとテストデータに分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # 標準化処理
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)

    # PyTorchテンソルに変換
    X_train_std = torch.tensor(X_train_std, dtype=torch.float32)
    X_test_std = torch.tensor(X_test_std, dtype=torch.float32)

    # 訓練データセットとテストデータセット
    trainset = TensorDataset(X_train_std, y_train)
    testset = TensorDataset(X_test_std, y_test)

    # 訓練データをランダムに分割
    partition_size = len(X_train_std) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # データローダ
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 検証データセットの大きさ
        len_train = len(ds) - len_val  # 残りの訓練データセットの大きさ
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))

        # 学習に訓練データをすべて利用したい場合(しっかりと検証したい場合はコメントアウト)
        ds_train, ds_val = ds, ds

        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)

    return trainloaders, valloaders, testloader

In [ ]:
# 歩行速度別にデータセットを分割
def load_datasets_by_speed():
    """
    データセットをロード
    """
    # 説明変数と目的変数に分割
    X = dataset_df.drop(["distance", "time"], axis=1).values
    y = ((dataset_df["time"] / dataset_df["distance"]) - (1 / dataset_df["speed"])).values

    # NumPy配列をPyTorchテンソルに変換
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    # サイズを(N, 1)に変更
    y = y.view(-1, 1)

    # 訓練データとテストデータに分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # 標準化処理
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)

    # PyTorchテンソルに変換
    X_train_std = torch.tensor(X_train_std, dtype=torch.float32)
    X_test_std = torch.tensor(X_test_std, dtype=torch.float32)

    # 訓練データセットとテストデータセット
    trainset = TensorDataset(X_train_std, y_train)
    testset = TensorDataset(X_test_std, y_test)

    # 標準化後の歩行速度リスト
    std_speed_list = []
    for i in range(len(trainset)):
        if (trainset[i][0][1] not in std_speed_list):
            std_speed_list.append(trainset[i][0][1].item())
    print(f"標準化後の歩行速度リスト\n  {std_speed_list}")

    # 歩行速度別のグループ
    speed_groups = {}
    for i in range(len(std_speed_list)):
        speed_groups[std_speed_list[i]] = []
    for i in range(len(trainset)):
        speed_groups[trainset[i][0][1].item()].append(trainset[i])

    # 歩行速度別にデータセットを作成
    trainset_by_speed = {}
    for speed, data in speed_groups.items():
        inputs, outputs = zip(*data)
        trainset_by_speed[speed] = TensorDataset(torch.stack(inputs), torch.stack(outputs))
    print("歩行速度別のデータ数")
    for i in range(len(std_speed_list)):
        print(f"  {len(trainset_by_speed[std_speed_list[i]])},", end="")

    print("\n各クライアントのデータ数")

    # 訓練データを歩行速度別に分割
    datasets = []
    for i in range(len(std_speed_list)):
        num_clients_by_speed = int(NUM_CLIENTS / len(std_speed_list))
        partition_size = len(trainset_by_speed[std_speed_list[i]]) // num_clients_by_speed
        lengths = [partition_size] * num_clients_by_speed
        remainder = len(trainset_by_speed[std_speed_list[i]]) - (partition_size * num_clients_by_speed)
        for j in range(remainder):
            lengths[j] += 1
        print(f"  {lengths},", end="")

        if (num_clients_by_speed == 1):
            datasets.append(trainset_by_speed[std_speed_list[i]])
        else:
            datasets_by_speed = random_split(trainset_by_speed[std_speed_list[i]], lengths, torch.Generator().manual_seed(42))
            for k in range(len(datasets_by_speed)):
                datasets.append(datasets_by_speed[k])

    # データローダ
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 検証データセットの大きさ
        len_train = len(ds) - len_val  # 残りの訓練データセットの大きさ
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))

        # 学習に訓練データをすべて利用したい場合(しっかりと検証したい場合はコメントアウト)
        ds_train, ds_val = ds, ds

        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)

    return trainloaders, valloaders, testloader

In [ ]:
if is_random == True:
    trainloaders, valloaders, testloader = load_datasets_random()
else:
    trainloaders, valloaders, testloader = load_datasets_by_speed()

標準化後の歩行速度リスト
  [-1.6115500926971436, 0.13179686665534973, 0.8291358947753906, -0.5655418038368225, 1.526474952697754, 1.1778054237365723, -0.21687224507331848, -1.262880802154541, -0.9142112731933594, 0.48046639561653137]
歩行速度別のデータ数
  656,  714,  720,  737,  770,  720,  694,  627,  683,  679,
各クライアントのデータ数
  [66, 66, 66, 66, 66, 66, 65, 65, 65, 65],  [72, 72, 72, 72, 71, 71, 71, 71, 71, 71],  [72, 72, 72, 72, 72, 72, 72, 72, 72, 72],  [74, 74, 74, 74, 74, 74, 74, 73, 73, 73],  [77, 77, 77, 77, 77, 77, 77, 77, 77, 77],  [72, 72, 72, 72, 72, 72, 72, 72, 72, 72],  [70, 70, 70, 70, 69, 69, 69, 69, 69, 69],  [63, 63, 63, 63, 63, 63, 63, 62, 62, 62],  [69, 69, 69, 68, 68, 68, 68, 68, 68, 68],  [68, 68, 68, 68, 68, 68, 68, 68, 68, 67],

In [ ]:
# # 訓練データの確認
# for data in trainloaders[0]:
#     X, y = data
#     print("Training Data - X:", X, "y:", y)

## 線形回帰モデル
- 説明変数：width, speed, crowd_level_2, crowd_level_3, crowd_level_4, crowd_level_5
- 目的変数：
$ \frac{1}{\text{distance}} \left( \text{time} - \frac{\text{distance}}{\text{speed}} \right) = \frac{\text{time}}{\text{distance}} - \frac{1}{\text{speed}} $

In [ ]:
input_dim = 6   # 説明変数の次元数
output_dim = 1  # 目的変数の次元数

In [ ]:
class Net(nn.Module):
    """ 線形回帰モデル """

    def __init__(self, input_dim: int, output_dim: int) -> None:
        """
        コンストラクタ

        Parameters
        ----------
        input_dim : int
            入力次元数
        output_dim : int
            出力次元数
        """
        super().__init__()
        self.l1 = nn.Linear(input_dim, output_dim)

        nn.init.constant_(self.l1.weight, 1.0)
        nn.init.constant_(self.l1.bias, 1.0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        順伝播

        Returns
        -------
        torch.Tensor
            1層目の出力
        """
        return self.l1(x)

## 学習関数

In [ ]:
def train(net: Net, trainloader: DataLoader, epochs: int, verbose: bool=False) -> np.ndarray:
    """
    訓練データセットで学習

    Parameters
    ----------
    net : DataLoader
        線形回帰モデル
    trainloader : DataLoader
        訓練ローダ
    epochs : int
        学習回数
    verbose : bool, default False
        学習の途中経過を出力するかしないか

    Returns
    -------
    history : np.ndarray
        学習記録
    """
    criterion_mse = nn.MSELoss()
    criterion_mae = nn.L1Loss()
    lr = 0.01
    optimizer = optim.SGD(net.parameters(), lr=lr)
    # optimizer = optim.Adam(net.parameters(), lr=lr)
    net.train()

    # 学習記録用
    history = np.zeros((0, 2))

    for epoch in range(epochs):
        epoch_mse_loss, epoch_mae_loss = 0.0, 0.0
        for X, y in trainloader:
            X, y = X.to(DEVICE), y.to(DEVICE)

            # 勾配値の初期化
            optimizer.zero_grad()

            # 予測計算（順伝播）
            outputs = net(X)

            # 損失計算（MSE）
            mse_loss = criterion_mse(outputs, y)

            # 損失計算（MAE）
            mae_loss = criterion_mae(outputs, y)

            # 勾配計算（逆伝播）
            mse_loss.backward()

            # パラメータの更新
            optimizer.step()

            # 評価指標
            epoch_mse_loss += mse_loss
            epoch_mae_loss += mae_loss

        epoch_mse_loss /= len(trainloader.dataset)
        epoch_mae_loss /= len(trainloader.dataset)

        # エポック10回ごとに途中経過を記録
        if (epoch % 10 == 0):
            history = np.vstack((history, np.array([epoch, epoch_mse_loss.item()])))

        if verbose:
            if (epoch % 10 == 0):
                print(f"Epoch {epoch}: MSE_loss = {epoch_mse_loss}, MAE_loss = {epoch_mae_loss}")

    return history

## テスト関数

In [ ]:
def test(net: Net, testloader: DataLoader) -> Tuple[float, float]:
    """
    テストデータセットで評価

    Parameters
    ----------
    net : Net
        線形回帰モデル
    trainloader : DataLoader
        テストローダ

    Returns
    -------
    mse_loss : float
        平均二乗誤差 (Mean Squared Error; MSE)
    mae_loss : float
        平均絶対誤差 (Mean Absolute Error; MAE)
    """
    criterion_mse = nn.MSELoss()
    criterion_mae = nn.L1Loss()
    mse_loss, mae_loss = 0.0, 0.0
    net.eval()

    with torch.no_grad():
        for X, y in testloader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            outputs = net(X)
            mse_loss += criterion_mse(outputs, y).item()
            mae_loss += criterion_mae(outputs, y).item()

    mse_loss /= len(testloader.dataset)
    mae_loss /= len(testloader.dataset)

    return mse_loss, mae_loss

## Flowerの実装

In [ ]:
def get_parameters(net: Net) -> List[np.ndarray]:
    """
    パラメータを取得

    Parameters
    ----------
    net : Net
        線形回帰モデル

    Returns
    -------
    List[np.ndarray]
        パラメータを格納したリスト
    """
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net: Net, parameters: List[np.ndarray]) -> None:
    """
    パラメータをモデルに適用

    Parameters
    ----------
    net : Net
        線形回帰モデル
    parameters : List[np.ndarray]
        パラメータを格納したリスト
    """
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    """ 連合学習を担うFlowerクライアント """

    def __init__(self, net: Net, trainloader: DataLoader, valloader: DataLoader) -> None:
        """
        コンストラクタ

        Parameters
        ----------
        net : Net
            線形回帰モデル
        trainloader : DataLoader
            訓練ローダ
        valloader : DataLoader
            検証ローダ
        """
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config) -> List[np.ndarray]:
        """
        ローカルモデルのパラメータを取得

        Returns
        -------
        List[np.ndarray]
            パラメータを格納したリスト
        """
        return get_parameters(self.net)

    def fit(self, parameters: List[np.ndarray], config) -> Tuple[List[np.ndarray], int, Dict]:
        """
        サーバからグローバルモデルのパラメータを受信
        ローカルの訓練データセットで学習
        更新モデルのパラメータをサーバに送信

        Parameters
        ----------
        parameters : List[np.ndarray]
            サーバから受信したパラメータを格納したリスト

        Returns
        -------
        List[np.ndarray]
            更新モデルのパラメータを格納したリスト
        int
            ローカルの訓練データセットの大きさ
        Dict
            空の辞書
        """
        set_parameters(self.net, parameters)
        history = train(self.net, self.trainloader, epochs=EPOCHS)
        return get_parameters(self.net), len(self.trainloader), {}
        # return get_parameters(self.net), len(self.trainloader), history

    def evaluate(self, parameters: List[np.ndarray], config) -> Tuple[float, int, Dict]:
        """
        サーバからグローバルモデルのパラメータを受信
        ローカルの検証データセットで評価
        評価結果をサーバに送信

        Parameters
        ----------
        parameters : List[np.ndarray]
            サーバから受信したパラメータを格納したリスト

        Returns
        -------
        mse_loss : float
            平均二乗誤差 (Mean Squared Error; MSE)
        int
            ローカルの検証データセットの大きさ
        Dict
            平均絶対誤差 (Mean Absolute Error; MAE)のキーと値
        """
        set_parameters(self.net, parameters)
        mse_loss, mae_loss = test(self.net, self.valloader)
        return float(mse_loss), len(self.valloader), {"MAE_loss": mae_loss}

In [ ]:
def client_fn(cid: str) -> FlowerClient:
    """
    1つのクライアント組織を表すFlowerクライアントの作成

    Parameters
    ----------
    cid : str
        クライアントID

    Returns
    -------
    FlowerClient
        クライアントIDによって識別されたFlowerクライアント
    """

    # モデルのロード
    net = Net(input_dim, output_dim).to(DEVICE)

    # ローカルデータセットのロード
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    return FlowerClient(net, trainloader, valloader)

## 評価関数

In [ ]:
# テストデータを混雑度別に分ける
X = dataset_df.drop(["distance", "time"], axis=1)
y = (dataset_df["time"] / dataset_df["distance"]) - (1 / dataset_df["speed"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

test_index_1 = X_test[(X_test["crowd_level_2"] == 0) & (X_test["crowd_level_3"] == 0) & (X_test["crowd_level_4"] == 0) & (X_test["crowd_level_5"] == 0)].index
test_index_2 = X_test[X_test["crowd_level_2"] == 1].index
test_index_3 = X_test[X_test["crowd_level_3"] == 1].index
test_index_4 = X_test[X_test["crowd_level_4"] == 1].index
test_index_5 = X_test[X_test["crowd_level_5"] == 1].index

X_test_1 = X_test.loc[test_index_1].values
y_test_1 = y_test.loc[test_index_1].values
X_test_2 = X_test.loc[test_index_2].values
y_test_2 = y_test.loc[test_index_2].values
X_test_3 = X_test.loc[test_index_3].values
y_test_3 = y_test.loc[test_index_3].values
X_test_4 = X_test.loc[test_index_4].values
y_test_4 = y_test.loc[test_index_4].values
X_test_5 = X_test.loc[test_index_5].values
y_test_5 = y_test.loc[test_index_5].values

X_test_1 = torch.tensor(X_test_1, dtype=torch.float32)
y_test_1 = torch.tensor(y_test_1, dtype=torch.float32)
X_test_2 = torch.tensor(X_test_2, dtype=torch.float32)
y_test_2 = torch.tensor(y_test_2, dtype=torch.float32)
X_test_3 = torch.tensor(X_test_3, dtype=torch.float32)
y_test_3 = torch.tensor(y_test_3, dtype=torch.float32)
X_test_4 = torch.tensor(X_test_4, dtype=torch.float32)
y_test_4 = torch.tensor(y_test_4, dtype=torch.float32)
X_test_5 = torch.tensor(X_test_5, dtype=torch.float32)
y_test_5 = torch.tensor(y_test_5, dtype=torch.float32)

y_test_1 = y_test_1.view(-1, 1)
y_test_2 = y_test_2.view(-1, 1)
y_test_3 = y_test_3.view(-1, 1)
y_test_4 = y_test_4.view(-1, 1)
y_test_5 = y_test_5.view(-1, 1)

X = dataset_df.drop(["distance", "time"], axis=1).values
y = (dataset_df["time"] / dataset_df["distance"]) - (1 / dataset_df["speed"]).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

sc = StandardScaler()
sc.fit(X_train)

X_test_std_1 = sc.transform(X_test_1)
X_test_std_2 = sc.transform(X_test_2)
X_test_std_3 = sc.transform(X_test_3)
X_test_std_4 = sc.transform(X_test_4)
X_test_std_5 = sc.transform(X_test_5)

X_test_std_1 = torch.tensor(X_test_std_1, dtype=torch.float32)
X_test_std_2 = torch.tensor(X_test_std_2, dtype=torch.float32)
X_test_std_3 = torch.tensor(X_test_std_3, dtype=torch.float32)
X_test_std_4 = torch.tensor(X_test_std_4, dtype=torch.float32)
X_test_std_5 = torch.tensor(X_test_std_5, dtype=torch.float32)

testset_1 = TensorDataset(X_test_std_1, y_test_1)
testloader_1 = DataLoader(testset_1, batch_size=BATCH_SIZE)
testset_2 = TensorDataset(X_test_std_2, y_test_2)
testloader_2 = DataLoader(testset_2, batch_size=BATCH_SIZE)
testset_3 = TensorDataset(X_test_std_3, y_test_3)
testloader_3 = DataLoader(testset_3, batch_size=BATCH_SIZE)
testset_4 = TensorDataset(X_test_std_4, y_test_4)
testloader_4 = DataLoader(testset_4, batch_size=BATCH_SIZE)
testset_5 = TensorDataset(X_test_std_5, y_test_5)
testloader_5 = DataLoader(testset_5, batch_size=BATCH_SIZE)

testloader_n = [testloader_1, testloader_2, testloader_3, testloader_4, testloader_5]

In [ ]:
def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, float]) -> Optional[Tuple[float, Dict[str, float]]]:
    """
    各ラウンド後にグローバルモデルを評価

    Parameters
    ----------
    server_round : int
        現在のラウンド
    parameters : fl.common.NDArrays
        パラメータ

    Returns
    mse_loss : float
        平均二乗誤差 (Mean Sauared Error; MSE)
    mae_loss_dict : Dict[str, float]
        平均絶対誤差 (Mean Absolute Error; MAE)のキーと値
    """
    net = Net(input_dim, output_dim).to(DEVICE)
    # 最新のパラメータでグローバルモデルを更新
    set_parameters(net, parameters)

    mse_loss, mae_loss = test(net, testloader)
    mse_loss_dict = {"MSE_loss": mse_loss}
    mae_loss_dict = {"MAE_loss": mae_loss}

    print(f"----- Round {server_round} server-side evaluation -----")
    print(f"All test data: MSE_loss = {mse_loss}, MAE_loss = {mae_loss}")

    for i in range(len(testloader_n)):
        mse_loss_, mae_loss_ = test(net, testloader_n[i])
        mse_loss_dict[f"MSE_loss_{i + 1}"] = mse_loss_
        mae_loss_dict[f"MAE_loss_{i + 1}"] = mae_loss_
        print(f"Crowd level {i + 1}: MSE_loss = {mse_loss_}, MAE_loss = {mae_loss_}")

    return mse_loss, mae_loss_dict

## シミュレーション

In [ ]:
# FedAvgアルゴリズムを用いた手法
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # 利用可能なクライアントの内、X%学習に使用
    fraction_evaluate=1.0,  # 利用可能なクライアントの内、X%評価に使用
    min_fit_clients=NUM_CLIENTS,  # クライアント数Xより少ない場合は学習しない
    min_evaluate_clients=NUM_CLIENTS,  # クライアント数Xより少ない場合は評価しない
    min_available_clients=NUM_CLIENTS,  # クライアント数Xが利用可能になるまで待つ
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(input_dim, output_dim))),  # モデルパラメータの初期値
    evaluate_fn=evaluate  # 評価関数
)

# FedAdamアルゴリズムを用いた手法
# strategy = fl.server.strategy.FedAdagrad(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(input_dim, output_dim))),
#     evaluate_fn=evaluate
# )

# 各クライアントの計算資源を指定
num_cpus = multiprocessing.cpu_count()
client_resources = {"num_cpus": num_cpus, "num_gpus": 0}
if DEVICE.type == "cuda":
    num_gpus = torch.cuda.device_count()
    client_resources["num_gpus"] = num_gpus

# 連合学習シミュレーションの開始
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=ROUNDS),
    strategy=strategy,
    client_resources=client_resources
)

INFO flwr 2024-01-18 10:45:01,476 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=30, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=30, round_timeout=None)
2024-01-18 10:45:05,815	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-01-18 10:45:07,770 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 32386100430.0, 'object_store_memory': 16193050214.0, 'CPU': 8.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 32386100430.0, 'object_store_memory': 16193050214.0, 'CPU': 8.0}
INFO flwr 2024-01-18 10:45:07,776 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-01-

----- Round 0 server-side evaluation -----
All test data: MSE_loss = 3.680575243328568, MAE_loss = 1.5870249590898553
Crowd level 1: MSE_loss = 3.3662569517754735, MAE_loss = 1.5237905299082732
Crowd level 2: MSE_loss = 3.889489654584127, MAE_loss = 1.6402422723103498
Crowd level 3: MSE_loss = 4.043093486310407, MAE_loss = 1.681275717254545
Crowd level 4: MSE_loss = 3.9249997778575585, MAE_loss = 1.6377711890214315


INFO flwr 2024-01-18 10:45:09,408 | server.py:94 | initial parameters (loss, other metrics): 3.680575243328568, {'MAE_loss': 1.5870249590898553, 'MAE_loss_1': 1.5237905299082732, 'MAE_loss_2': 1.6402422723103498, 'MAE_loss_3': 1.681275717254545, 'MAE_loss_4': 1.6377711890214315, 'MAE_loss_5': 1.4529072625502464}
INFO:flwr:initial parameters (loss, other metrics): 3.680575243328568, {'MAE_loss': 1.5870249590898553, 'MAE_loss_1': 1.5237905299082732, 'MAE_loss_2': 1.6402422723103498, 'MAE_loss_3': 1.681275717254545, 'MAE_loss_4': 1.6377711890214315, 'MAE_loss_5': 1.4529072625502464}
INFO flwr 2024-01-18 10:45:09,415 | server.py:104 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2024-01-18 10:45:09,420 | server.py:222 | fit_round 1: strategy sampled 100 clients (out of 100)
DEBUG:flwr:fit_round 1: strategy sampled 100 clients (out of 100)


Crowd level 5: MSE_loss = 3.1872224137294847, MAE_loss = 1.4529072625502464


(pid=25261) 2024-01-18 10:45:09.487923: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=25261) 2024-01-18 10:45:09.488023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=25261) 2024-01-18 10:45:09.489487: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=25261) 2024-01-18 10:45:10.930534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEBUG flwr 2024-01-18 10:47:24,944 | server.py:236 | fit_round 1 received 100 results and 0 failures
DEBUG:flwr:fit_round 1 received 100 results and 0 failures
WARNING flwr 2024-01-18 10:47:24,

...

----- Round 30 server-side evaluation -----
All test data: MSE_loss = 0.007921869019746208, MAE_loss = 0.06269616221180574
Crowd level 1: MSE_loss = 0.002082697654570915, MAE_loss = 0.038703841409242795
Crowd level 2: MSE_loss = 0.0024543435044824626, MAE_loss = 0.037876873057987225
Crowd level 3: MSE_loss = 0.006221789057295664, MAE_loss = 0.05091155309451469


INFO flwr 2024-01-18 11:51:45,848 | server.py:125 | fit progress: (30, 0.007921869019746208, {'MAE_loss': 0.06269616221180574, 'MAE_loss_1': 0.038703841409242795, 'MAE_loss_2': 0.037876873057987225, 'MAE_loss_3': 0.05091155309451469, 'MAE_loss_4': 0.07350698373023715, 'MAE_loss_5': 0.11458493340981836}, 3996.427687542)
INFO:flwr:fit progress: (30, 0.007921869019746208, {'MAE_loss': 0.06269616221180574, 'MAE_loss_1': 0.038703841409242795, 'MAE_loss_2': 0.037876873057987225, 'MAE_loss_3': 0.05091155309451469, 'MAE_loss_4': 0.07350698373023715, 'MAE_loss_5': 0.11458493340981836}, 3996.427687542)
DEBUG flwr 2024-01-18 11:51:45,851 | server.py:173 | evaluate_round 30: strategy sampled 100 clients (out of 100)
DEBUG:flwr:evaluate_round 30: strategy sampled 100 clients (out of 100)


Crowd level 4: MSE_loss = 0.010581959500223767, MAE_loss = 0.07350698373023715
Crowd level 5: MSE_loss = 0.018737123566089836, MAE_loss = 0.11458493340981836


DEBUG flwr 2024-01-18 11:51:50,631 | server.py:187 | evaluate_round 30 received 100 results and 0 failures
DEBUG:flwr:evaluate_round 30 received 100 results and 0 failures
INFO flwr 2024-01-18 11:51:50,635 | server.py:153 | FL finished in 4001.2147859449997
INFO:flwr:FL finished in 4001.2147859449997
INFO flwr 2024-01-18 11:51:50,639 | app.py:226 | app_fit: losses_distributed [(1, 0.47303824971917136), (2, 0.14854075290008137), (3, 0.05467069715121209), (4, 0.024463046164091708), (5, 0.01387492423803151), (6, 0.010169612533759696), (7, 0.008786195011459214), (8, 0.008217556334804563), (9, 0.00800197253624286), (10, 0.007944977106355902), (11, 0.007910293677439846), (12, 0.007899971083699116), (13, 0.007890833699062074), (14, 0.007890679118713178), (15, 0.007888478214912955), (16, 0.007887687232104297), (17, 0.007887037340011775), (18, 0.007888496891251456), (19, 0.007886700389513217), (20, 0.007887957779454564), (21, 0.007888529901025839), (22, 0.00788777019411751), (23, 0.007889262838

History (loss, distributed):
	round 1: 0.47303824971917136
	round 2: 0.14854075290008137
	round 3: 0.05467069715121209
	round 4: 0.024463046164091708
	round 5: 0.01387492423803151
	round 6: 0.010169612533759696
	round 7: 0.008786195011459214
	round 8: 0.008217556334804563
	round 9: 0.00800197253624286
	round 10: 0.007944977106355902
	round 11: 0.007910293677439846
	round 12: 0.007899971083699116
	round 13: 0.007890833699062074
	round 14: 0.007890679118713178
	round 15: 0.007888478214912955
	round 16: 0.007887687232104297
	round 17: 0.007887037340011775
	round 18: 0.007888496891251456
	round 19: 0.007886700389513217
	round 20: 0.007887957779454564
	round 21: 0.007888529901025839
	round 22: 0.00788777019411751
	round 23: 0.007889262838169894
	round 24: 0.007887661136051905
	round 25: 0.007888193490344983
	round 26: 0.007887953751201066
	round 27: 0.007888715682173765
	round 28: 0.007889073222211315
	round 29: 0.007887202297759435
	round 30: 0.007887655755136934
History (loss, centralized

## 評価

1m歩いた時の平均時間誤差 (MAE) [s/m]

- ランダムに分割

| クライアント数 | 収束ラウンド数 | 全体 | 混雑度1 | 混雑度2 |  混雑度3 |  混雑度4 |  混雑度5 |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 10 | 2 | 0.0619112033 | 0.0372089321 | 0.0365712017 | 0.0507407992 | 0.0737848683 | 0.1133870780 |
| 50 | 3 | 0.0620158370 | 0.0367395795 | 0.0361763794 | 0.0496458076 | 0.0759601122 | 0.1137922705 |
| 100 | 5 | 0.0623555531 | 0.0377113196 | 0.0369542538 | 0.0507650156 | 0.0737137413 | 0.1147774621 |
| 500 | 30 | 0.0624800370 | 0.0375856798 | 0.0368313923 | 0.0511188377 | 0.0738422583 | 0.1151743999 |
| 1000 | 100 | 0.0625261947 | 0.0380095562 | 0.0372345823 | 0.0510124069 | 0.0738623510 | 0.1146447972 |

<br>

- 歩行速度別に分割

| クライアント数 | 収束ラウンド数 | 全体 | 混雑度1 | 混雑度2 |  混雑度3 |  混雑度4 |  混雑度5 |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 10 | 10 | 0.0625724085 | 0.0379385760 | 0.0372291169 | 0.0499230392 | 0.0749120427 | 0.1150485299 |
| 50 | 15 | 0.0627550964 | 0.0388325309 | 0.0378466156 | 0.0506888826 | 0.0739609882 | 0.1145591556 |
| 100 | 20 | 0.0628205000 | 0.0389990176 | 0.0381010128 | 0.0509919797 | 0.0737414638 | 0.1143638297 |
| 500 | 30 | 0.0628307263 | 0.0382393137 | 0.0374111600 | 0.0512425374 | 0.0748514561 | 0.1145554759 |
| 1000 | 75 | 0.0628108941 | 0.0380571892 | 0.0372053137 | 0.0511329156 | 0.0748489026 | 0.1149706355 |